In [45]:
import config
import requests
import json
from nltk.tokenize import RegexpTokenizer
from collections import defaultdict
import pandas as pd


In [2]:
Authentic_api_key = config.Authentic_api_key

In [3]:
# https://authenticjobs.com/api/docs

In [4]:
base_url = 'https://authenticjobs.com/api/'
params = {'api_key': Authentic_api_key,
          'format': 'json',
          }

In [5]:
# I'm not sure how to comply with requirement #1 from here:
# https://authenticjobs.com/api/docs#security
# given that python dicts are inherently unordered

In [25]:
params['method'] = 'aj.types.getList'
types = requests.get(base_url, params).json()
print(json.dumps(types,  indent=3))

In [26]:
params['method'] = 'aj.categories.getList'
categories = requests.get(base_url, params).json()
#print(json.dumps(categories,  indent=3))

In [36]:
params['method'] = 'aj.jobs.getCompanies'
companies = requests.get(base_url, params).json()
#print(json.dumps(companies,  indent=3))

{
   "companies": {
      "company": [
         {
            "id": "accomplice",
            "name": "Accomplice",
            "url": "https://accpl.co/",
            "logo": "https://d2fcz5no062gar.cloudfront.net/assets/images/defaults/company-blank.png",
            "tagline": "Driving Digital & Organizational Success",
            "location": {
               "id": "austintxus",
               "name": "Woodbridge, VA (Metro DC)",
               "city": "Austin",
               "country": "US",
               "lat": "30.2672",
               "lng": "-97.7431",
               "state": "TX"
            },
            "count": 1
         },
         {
            "id": "advicepay",
            "name": "AdvicePay",
            "url": "http://advicepay.com",
            "logo": "https://d2fcz5no062gar.cloudfront.nethttps://authenticjobs.s3.amazonaws.com/uploads/logos/2901db334bdbf06c0b30174eb5148630/thumb/Screen Shot 2018-06-20 at 11.12.58 AM.png",
            "tagline": "Payment for Fin

In [246]:
params['method'] = 'aj.jobs.search'
params['keywords'] = 'data'
params['page'] = '4'
search_results_page = requests.get(base_url, params).json()

In [7]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [9]:
from IPython.core.display import HTML

# Print job descriptions without html
for l in search_results_page['listings']['listing']:
    print(l['id'])
    print(strip_tags(l['description']))

# Display job descriptions formatted with html
for l in search_results_page['listings']['listing']:
    display(HTML(l['description']))

30669
Organization
 
Rooted in the guiding principles of President George W. Bush and Mrs. Laura Bush, the George W. Bush Presidential Center engages communities in the United States and around the world by cultivating leaders and advancing policies to solve today’s most pressing challenges. The Bush Center is home to the George W. Bush Presidential Library and Museum, which is operated by the National Archives and Records Administration, and the George W. Bush Institute, the non-partisan, public-policy arm of the Bush Center. The Bush Center is located on the campus of SMU in Dallas, Texas, and also includes a 15-acre park, a full-service restaurant, Café 43, and a museum store.
 
Housed within the George W. Bush Presidential Center, the George W. Bush Institute is an action-oriented, nonpartisan policy organization with the mission of cultivating leaders and advancing policies to solve today’s most pressing challenges. It raises current and thought-provoking issues and builds program

In [28]:
# Read a list of skill words from a file
# import csv
# single_word_skills = []
# with open('single_word_skills.csv', 'r', newline='') as csvfile:
#     csv_reader = csv.reader(csvfile)
#     for row in csv_reader:
#         single_word_skills.append(str(*row).strip())
# single_word_skills = set(single_word_skills)

In [242]:
# Read a list of skill phrases from a file
skill_phrases = []
with open('skill_phrases.csv', 'r', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        skill_phrases.append(str(*row).strip())
skill_phrases = set(skill_phrases)

In [244]:
#separate each skill phrase into a list of its words
tokenizer = RegexpTokenizer(r'[a-zA-Z0-9#+-]+')
skill_phrase_wl = [tokenizer.tokenize(skill_phrase) for skill_phrase in skill_phrases]

In [247]:
html_start_tag = '<font color="red">'
html_end_tag = '</font>'

# For each listing returned
for l in search_results_page['listings']['listing']:
    description = l['description']
    # tokenize the text of the description, with spans
    tokenizer = RegexpTokenizer(r'[a-zA-Z0-9#+-]+')
    span_generator = tokenizer.span_tokenize(description)
    spans = [span for span in span_generator]
    tokens = [description[span[0]:span[1]] for span in spans]

#     # create a dictionary of the words, with spans as the values
#     # and another dictionary with the same keys, with the word indexes as the values
    char_span = defaultdict(list)
    word_index = defaultdict(list)
    for i, (k, span) in enumerate(zip(tokens, spans)):
        char_span[k].append(span)
        word_index[k].append(i)
    # is this useful?
    df = pd.DataFrame({'Character Index Spans': pd.Series(char_span), 'Word Indexes': pd.Series(word_index)})

    highlight_spans = []   
    for skill_phrase in skill_phrase_wl:
        if word_index.get(skill_phrase[0]):
            for i, occurence in enumerate(word_index.get(skill_phrase[0])):
                if all((skill_phrase[j] == tokens[j+occurence]) for j in range(len(skill_phrase))):
                    highlight_span = (spans[occurence][0], spans[occurence + len(skill_phrase) - 1][1])
                    highlight_spans.append (highlight_span)

# # look up the words in our skill list in the dictionary.  List the findings as spans to be highlighted
#     for skill in single_word_skills:
#         highlight_spans += char_span[skill]

    # Sort the spans to be highlighted
    highlight_spans.sort()

    # Insert html tags to highlight the keywords
    highlighted = ''
    cursor = 0
    for span in highlight_spans:
        if (span[0] > cursor): # go forwards only, not backwards 
            if (cursor>0):
                highlighted += html_end_tag
            highlighted += description[cursor:span[0]] + \
                            html_start_tag + \
                            description[span[0]:span[1]]
        elif (span[1] > cursor):
            highlighted += description[cursor:span[1]]
        cursor = span[1]
    highlighted += html_end_tag + description[cursor:]
    display(HTML(highlighted))   

In [239]:
import csv
with open('skill_phrases_out.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    for skill_phrase in skill_phrases:
        csv_writer.writerow([skill_phrase])